In [1]:
import sys
sys.path.append('../../..')
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_orderbooks_from_file, get_binance_orderbook
from hydradx.model.processing import get_omnipool_data_from_file, load_centralized_market
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb

from pprint import pprint

import json


Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = True

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [4]:
# load Omnipool
RPC="wss://rpc.hydradx.cloud"
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, archive=False)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("data/input/")
    
# pprint(tokens)
init_state = OmnipoolState(
    tokens=tokens,
    lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list},
    asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list},
    preferred_stablecoin='USDT',
)

In [5]:
# load centralized markets
# arb_file = "arbconfig.txt"
arb_file = "arbconfig2.txt"

with open('config/' + arb_file, 'r') as json_file:
    cfg = json.load(json_file)

for d in cfg:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])
    d['tkn_pair'] = (asset_map[d['tkn_ids'][0]], asset_map[d['tkn_ids'][1]])

cex_fees = {
    'kraken': 0.0016,
    'binance': 0.0010
}

cex_dict = {}
for exchange in ('kraken', 'binance'):
    if get_live_markets:
        cex_dict[exchange] = load_centralized_market(
            config_filename=arb_file, 
            exchange_name=exchange, 
            trade_fee=cex_fees[exchange]
        )
    else:
        cex_dict[exchange] = CentralizedMarket(
            orderbook=get_orderbooks_from_file("data/input/")[exchange],
            unique_id=exchange,
            trade_fee=cex_fees[exchange]
        )

In [6]:
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state1 = init_state.copy()
all_swaps1 = arb.get_arb_swaps_simple(op_state1, cex_dict1, cfg)

# pprint(all_swaps1)
# pprint(len(all_swaps1))

cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state2 = init_state.copy()
all_swaps2 = arb.get_arb_swaps(op_state2, cex_dict2, cfg)
# print("strategy 2")
# pprint(all_swaps2)
# pprint(len(all_swaps2))
# print(cfg)

In [7]:
state1 = init_state.copy()
state2 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list}
for exchange in cex_dict:
    for asset in cex_dict[exchange].asset_list:
        if asset not in holdings:
            holdings[asset] = 1000000000
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}

arb.execute_arb(state1, cex_dict1, agent1, all_swaps1)
arb.execute_arb(state2, cex_dict2, agent2, all_swaps2)

asset_map = {
    'WETH': 'ETH',
    'XETH': 'ETH',
    'XXBT': 'BTC',
    'WBTC': 'BTC',
    'ZUSD': 'USD',
    'USDT': 'USD',
    'USDC': 'USD',
    'DAI': 'USD',
    'USDT001': 'USD',
    'DAI001': 'USD',
    'WETH001': 'ETH',
    'WBTC001': 'BTC',
    'iBTC': 'BTC',
    'XBT': 'BTC',
}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb.calculate_profit(init_agent, agent2, asset_map)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Binance spot")
print(cex_dict1['binance'].value_assets(profit1, asset_map))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Binance spot")
print(cex_dict2['binance'].value_assets(profit2, asset_map))

# try rerouting to see if we can squeeze some more profit out of it
agent3 = init_agent.copy()
state3 = init_state.copy()
cex_dict3 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
all_swaps3 = arb.combine_swaps(
    dex=state3,
    cex=cex_dict3,
    agent=agent3,
    all_swaps=all_swaps2,
    asset_map=asset_map
)
arb.execute_arb(state3, cex_dict3, agent3, all_swaps3)
profit3 = arb.calculate_profit(init_agent, agent3, asset_map)
profit3_omnipool = arb.calculate_profit(init_agent, agent3, asset_map_omnipool)

print("strategy 3 profit")
print(profit3)
print("strategy 3 profit at Binance spot")
print(cex_dict3['binance'].value_assets(profit3, asset_map))


strategy 1 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 34.95166027545929, 'ETH': 0.0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0.0}
strategy 1 profit at Binance spot
34.95166027545929
strategy 2 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 38.40719282627106, 'ETH': 0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0.0}
strategy 2 profit at Binance spot
38.40719282627106
[{'exchange': 'dex', 'trade': 'buy', 'buy_asset': 'iBTC', 'sell_asset': 'DAI001', 'price': 36400.0, 'amount': 0.00056, 'max_sell': 19.126320900142193}, {'exchange': 'binance', 'trade': 'sell', 'buy_asset': 'DAI', 'sell_asset': 'BTC', 'price': 36381.8, 'amount': 0.00056}, {'exchange': 'dex', 'trade': 'buy', 'buy_asset': 'iBTC', 'sell_asset': 'DAI001', 'price': 36393.0, 'amount': 0.03197532747650146, 'max_sell': 1126.769566449344}, {'exchange': 'binance', 'trade': 'sell', 'buy_asset': 'DAI', 'sell_asset': 'BTC', 'price': 36374.8035, '

NameError: name 'asset_map_omnipool' is not defined